In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random
from googletrans import Translator
from pandas import Series
from pandas import DataFrame
%matplotlib inline


In [2]:
item_category_df = pd.read_csv("data/item_categories.csv")
items_df = pd.read_csv("data/items.csv")
shops_df = pd.read_csv("data/shops.csv")
sample_df = pd.read_csv("data/sample_submission.csv")
train_df = pd.read_csv("data/sales_train_v2.csv")
test_df = pd.read_csv("data/test.csv")

# date-> day, month, year

In [3]:
f = lambda x: x.split(".")
train_df['date'] =list(map(f, train_df['date']))

In [4]:
f = lambda x: x[0]
train_df['day'] =list(map(f, train_df['date']))

In [5]:
f = lambda x: x[1]
train_df['month'] =list(map(f, train_df['date']))

In [6]:
f = lambda x: x[2]
train_df['year'] =list(map(f, train_df['date']))

In [7]:
train_df = train_df.drop(['date'],axis=1)

In [8]:
train_df.head()

,date_block_num,shop_id,item_id,item_price,item_cnt_day,day,month,year
0,0,59,22154,999.00,1.0,02,01,2013
1,0,25,2552,899.00,1.0,03,01,2013
2,0,25,2552,899.00,-1.0,05,01,2013
3,0,25,2554,1709.05,1.0,06,01,2013
4,0,25,2555,1099.00,1.0,15,01,2013


## 번역 (러시아어 -> 영어) -> tf-idf

In [9]:
trans = Translator()
def translate(column):
    en_list=[]
    for word in column:
        en = trans.translate(word, dest='en')
        en_list.append(en.text)
    return en_list

In [10]:
shops_df["shop_name"] = translate(shops_df["shop_name"])
shops_df.head()

,shop_name,shop_id
0,"Yakutsk Ordzhonikidze, 56 francs",0
1,"Yakutsk TC ""Central"" franc",1
2,"Adygea Shopping Center ""Mega""",2
3,"Balashikha TRK ""October-Kinomir""",3
4,"Volzhsky shopping center ""Volga Mall""",4


In [11]:
en_shop_name = np.asarray(shops_df.shop_name)

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,4), min_df=0.03, max_df = 0.95)
shop_name_model = vectorizer.fit_transform(en_shop_name).toarray()

C:\Users\my\Miniconda3\envs\ml_python\lib\site-packages\sklearn\feature_extraction\text.py:1089: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


In [13]:
shop_X =[]
for i in range(len(en_shop_name)):
    shop_X.append(shop_name_model[i])

In [14]:
shops_df["shop_name_tfidf"] = shop_X
shops_df.head()

,shop_name,shop_id,shop_name_tfidf
0,"Yakutsk Ordzhonikidze, 56 francs",0,"[0.416609635057569, 0.0, 0.0, 0.0, 0.0, 0.0, 0..."
1,"Yakutsk TC ""Central"" franc",1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.53929388..."
2,"Adygea Shopping Center ""Mega""",2,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.4538104235202..."
3,"Balashikha TRK ""October-Kinomir""",3,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,"Volzhsky shopping center ""Volga Mall""",4,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.4320054028830..."


In [15]:
for i in range(len(shops_df)):
    for j in range(len(shops_df['shop_name_tfidf'][0])):
        shops_df['shop_name_tfidf' + str(j)]= shops_df['shop_name_tfidf'][i][j]

In [16]:
shops_df = shops_df.drop(['shop_name_tfidf'],axis=1)
shops_df.head()

,shop_name,shop_id,shop_name_tfidf0,shop_name_tfidf1,shop_name_tfidf2,shop_name_tfidf3,shop_name_tfidf4,shop_name_tfidf5,shop_name_tfidf6,shop_name_tfidf7,...,shop_name_tfidf65,shop_name_tfidf66,shop_name_tfidf67,shop_name_tfidf68,shop_name_tfidf69,shop_name_tfidf70,shop_name_tfidf71,shop_name_tfidf72,shop_name_tfidf73,shop_name_tfidf74
0,"Yakutsk Ordzhonikidze, 56 francs",0,0.0,0.0,0.0,0.0,0.0,0.0,0.546225,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"Yakutsk TC ""Central"" franc",1,0.0,0.0,0.0,0.0,0.0,0.0,0.546225,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"Adygea Shopping Center ""Mega""",2,0.0,0.0,0.0,0.0,0.0,0.0,0.546225,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,"Balashikha TRK ""October-Kinomir""",3,0.0,0.0,0.0,0.0,0.0,0.0,0.546225,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"Volzhsky shopping center ""Volga Mall""",4,0.0,0.0,0.0,0.0,0.0,0.0,0.546225,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
shops_df.to_pickle("./data/shops_df.pickle")

In [18]:
item_category_df["item_category_name"] = translate(item_category_df["item_category_name"])
item_category_df.head()

,item_category_name,item_category_id
0,PC - Headsets / Headphones,0
1,Accessories - PS2,1
2,Accessories - PS3,2
3,Accessories - PS4,3
4,Accessories - PSP,4


In [19]:
en_item_category_name = np.asarray(item_category_df.item_category_name)

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,4), min_df=0.03, max_df = 0.95)
item_category_name_model = vectorizer.fit_transform(en_item_category_name).toarray()

C:\Users\my\Miniconda3\envs\ml_python\lib\site-packages\sklearn\feature_extraction\text.py:1089: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


In [21]:
item_category_X =[]
for i in range(len(en_item_category_name)):
    item_category_X.append(item_category_name_model[i])

In [22]:
item_category_df["item_category_tfidf"] = item_category_X
item_category_df.head()

,item_category_name,item_category_id,item_category_tfidf
0,PC - Headsets / Headphones,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,Accessories - PS2,1,"[0.0, 0.6247350911517717, 0.0, 0.0, 0.0, 0.0, ..."
2,Accessories - PS3,2,"[0.0, 0.6247350911517717, 0.0, 0.0, 0.0, 0.0, ..."
3,Accessories - PS4,3,"[0.0, 0.6247350911517717, 0.0, 0.0, 0.0, 0.0, ..."
4,Accessories - PSP,4,"[0.0, 0.6247350911517717, 0.0, 0.0, 0.0, 0.0, ..."


In [23]:
for i in range(len(item_category_df)):
    for j in range(len(item_category_df['item_category_tfidf'][0])):
        item_category_df['item_category_tfidf' + str(j)]= item_category_df['item_category_tfidf'][i][j]

In [24]:
item_category_df = item_category_df.drop(['item_category_tfidf'],axis=1)
item_category_df.head()

,item_category_name,item_category_id,item_category_tfidf0,item_category_tfidf1,item_category_tfidf2,item_category_tfidf3,item_category_tfidf4,item_category_tfidf5,item_category_tfidf6,item_category_tfidf7,...,item_category_tfidf28,item_category_tfidf29,item_category_tfidf30,item_category_tfidf31,item_category_tfidf32,item_category_tfidf33,item_category_tfidf34,item_category_tfidf35,item_category_tfidf36,item_category_tfidf37
0,PC - Headsets / Headphones,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Accessories - PS2,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Accessories - PS3,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Accessories - PS4,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Accessories - PSP,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
item_category_df.to_pickle("./data/item_category_df.pickle")

In [26]:
items_df.head()

,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


In [27]:
rus_item_name = np.asarray(items_df.item_name)

In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,4), min_df=0.03, max_df = 0.95)
rus_item_name_model = vectorizer.fit_transform(rus_item_name).toarray()

C:\Users\my\Miniconda3\envs\ml_python\lib\site-packages\sklearn\feature_extraction\text.py:1089: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


In [29]:
item_X =[]
for i in range(len(rus_item_name)):
    item_X.append(rus_item_name_model[i])

In [30]:
items_df["item_tfidf"] = item_X
items_df.head()

,item_name,item_id,item_category_id,item_tfidf
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,!ABBYY FineReader 12 Professional Edition Full...,1,76,"[0.0, 0.0, 0.0, 0.0, 0.4825704672175713, 0.0, ..."
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,***КОРОБКА (СТЕКЛО) D,4,40,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [31]:
for i in range(len(items_df)):
    for j in range(len(items_df['item_tfidf'][0])):
        items_df['item_tfidf' + str(j)]= items_df['item_tfidf'][i][j]

In [32]:
items_df = items_df.drop(['item_tfidf'],axis=1)
items_df.head()

,item_name,item_id,item_category_id,item_tfidf0,item_tfidf1,item_tfidf2,item_tfidf3,item_tfidf4,item_tfidf5,item_tfidf6,...,item_tfidf13,item_tfidf14,item_tfidf15,item_tfidf16,item_tfidf17,item_tfidf18,item_tfidf19,item_tfidf20,item_tfidf21,item_tfidf22
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,!ABBYY FineReader 12 Professional Edition Full...,1,76,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,***КОРОБКА (СТЕКЛО) D,4,40,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [33]:
items_df.to_pickle("./data/items_df.pickle")

# merge

In [34]:
df =pd.merge(train_df, shops_df, on='shop_id')
df.head()

,date_block_num,shop_id,item_id,item_price,item_cnt_day,day,month,year,shop_name,shop_name_tfidf0,...,shop_name_tfidf65,shop_name_tfidf66,shop_name_tfidf67,shop_name_tfidf68,shop_name_tfidf69,shop_name_tfidf70,shop_name_tfidf71,shop_name_tfidf72,shop_name_tfidf73,shop_name_tfidf74
0,0,59,22154,999.0,1.0,02,01,2013,"Yaroslavl Shopping center ""Altair""",0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,59,22151,399.0,1.0,10,01,2013,"Yaroslavl Shopping center ""Altair""",0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,59,5603,699.0,1.0,04,01,2013,"Yaroslavl Shopping center ""Altair""",0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,59,5587,199.0,2.0,19,01,2013,"Yaroslavl Shopping center ""Altair""",0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,59,5613,5571.0,1.0,31,01,2013,"Yaroslavl Shopping center ""Altair""",0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [35]:
df =pd.merge(df, items_df, on='item_id')
df.head()

,date_block_num,shop_id,item_id,item_price,item_cnt_day,day,month,year,shop_name,shop_name_tfidf0,...,item_tfidf13,item_tfidf14,item_tfidf15,item_tfidf16,item_tfidf17,item_tfidf18,item_tfidf19,item_tfidf20,item_tfidf21,item_tfidf22
0,0,59,22154,999.0,1.0,02,01,2013,"Yaroslavl Shopping center ""Altair""",0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,25,22154,999.0,1.0,02,01,2013,"Moscow TRK ""Atrium""",0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,25,22154,999.0,1.0,03,01,2013,"Moscow TRK ""Atrium""",0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,25,22154,999.0,1.0,20,01,2013,"Moscow TRK ""Atrium""",0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,25,22154,999.0,1.0,23,01,2013,"Moscow TRK ""Atrium""",0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [36]:
df =pd.merge(df, item_category_df, on='item_category_id')
df.head()

,date_block_num,shop_id,item_id,item_price,item_cnt_day,day,month,year,shop_name,shop_name_tfidf0,...,item_category_tfidf28,item_category_tfidf29,item_category_tfidf30,item_category_tfidf31,item_category_tfidf32,item_category_tfidf33,item_category_tfidf34,item_category_tfidf35,item_category_tfidf36,item_category_tfidf37
0,0,59,22154,999.0,1.0,02,01,2013,"Yaroslavl Shopping center ""Altair""",0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,25,22154,999.0,1.0,02,01,2013,"Moscow TRK ""Atrium""",0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,25,22154,999.0,1.0,03,01,2013,"Moscow TRK ""Atrium""",0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,25,22154,999.0,1.0,20,01,2013,"Moscow TRK ""Atrium""",0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,25,22154,999.0,1.0,23,01,2013,"Moscow TRK ""Atrium""",0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [37]:
df = df.drop(['item_name'],axis=1)
df = df.drop(['item_category_name'],axis=1)

In [38]:
df.head()

,date_block_num,shop_id,item_id,item_price,item_cnt_day,day,month,year,shop_name,shop_name_tfidf0,...,item_category_tfidf28,item_category_tfidf29,item_category_tfidf30,item_category_tfidf31,item_category_tfidf32,item_category_tfidf33,item_category_tfidf34,item_category_tfidf35,item_category_tfidf36,item_category_tfidf37
0,0,59,22154,999.0,1.0,02,01,2013,"Yaroslavl Shopping center ""Altair""",0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,25,22154,999.0,1.0,02,01,2013,"Moscow TRK ""Atrium""",0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,25,22154,999.0,1.0,03,01,2013,"Moscow TRK ""Atrium""",0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,25,22154,999.0,1.0,20,01,2013,"Moscow TRK ""Atrium""",0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,25,22154,999.0,1.0,23,01,2013,"Moscow TRK ""Atrium""",0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# add location feature

## 동,서,남 방위와 온라인, 무역 등의 shop들로 구성됨
## 지역 위치에 따른 판매량 분석

In [39]:
loc_shop = df["shop_name"]
len(loc_shop)

2935849

In [40]:
loc = []
for i in loc_shop:
    splited_loc = i.split()
    loc.append(splited_loc[0])

In [41]:
df['location'] = loc

In [42]:
west= ['Adygea','Balashikha', 'Chekhov','Kaluga','Kazan', 'Krasnoyarsk' ,'Kursk', 'Moscow' ,'Yaroslavl',
        'Mytischi', 'N.Novgorod', 'Nizhny','RostovNaDon',  'RostovNaDonu', 'Samara' ,'SPb' ,'Sergiev', 
        'Surgut', 'Tyumen', 'Ufa', 'Vologda', 'Volzhsky', 'Voronezh' , 'Zhukovsky','Colosseum', 'Khimki']
east =  ['Yakutsk']
south =  ['Novosibirsk','Omsk', 'Tomsk']

In [43]:
s1 = Series(data = 'west', index= west)
west = s1.to_dict()

s2 = Series(data = 'east'
   , index= east)
east = s2.to_dict()

s3 = Series(data = 'south'
   , index= south)
south = s3.to_dict()

In [44]:
df["location"].replace(to_replace=west.keys(), value=west.values(), inplace=True)
df["location"].replace(to_replace=east.keys(), value=east.values(), inplace=True)
df["location"].replace(to_replace=south.keys(), value=south.values(), inplace=True)
df.head()

,date_block_num,shop_id,item_id,item_price,item_cnt_day,day,month,year,shop_name,shop_name_tfidf0,...,item_category_tfidf29,item_category_tfidf30,item_category_tfidf31,item_category_tfidf32,item_category_tfidf33,item_category_tfidf34,item_category_tfidf35,item_category_tfidf36,item_category_tfidf37,location
0,0,59,22154,999.0,1.0,02,01,2013,"Yaroslavl Shopping center ""Altair""",0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,west
1,0,25,22154,999.0,1.0,02,01,2013,"Moscow TRK ""Atrium""",0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,west
2,0,25,22154,999.0,1.0,03,01,2013,"Moscow TRK ""Atrium""",0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,west
3,0,25,22154,999.0,1.0,20,01,2013,"Moscow TRK ""Atrium""",0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,west
4,0,25,22154,999.0,1.0,23,01,2013,"Moscow TRK ""Atrium""",0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,west


In [45]:
dummy_loc_df =pd.get_dummies(df['location'])
dummy_loc_df.head()

,Digital,Online,Outbound,east,south,west
0,0,0,0,0,0,1
1,0,0,0,0,0,1
2,0,0,0,0,0,1
3,0,0,0,0,0,1
4,0,0,0,0,0,1


In [46]:
df = pd.concat([df, dummy_loc_df],axis=1)
df = df.drop(['location'],axis=1)
df.head()

,date_block_num,shop_id,item_id,item_price,item_cnt_day,day,month,year,shop_name,shop_name_tfidf0,...,item_category_tfidf34,item_category_tfidf35,item_category_tfidf36,item_category_tfidf37,Digital,Online,Outbound,east,south,west
0,0,59,22154,999.0,1.0,02,01,2013,"Yaroslavl Shopping center ""Altair""",0.0,...,0.0,0.0,0.0,0.0,0,0,0,0,0,1
1,0,25,22154,999.0,1.0,02,01,2013,"Moscow TRK ""Atrium""",0.0,...,0.0,0.0,0.0,0.0,0,0,0,0,0,1
2,0,25,22154,999.0,1.0,03,01,2013,"Moscow TRK ""Atrium""",0.0,...,0.0,0.0,0.0,0.0,0,0,0,0,0,1
3,0,25,22154,999.0,1.0,20,01,2013,"Moscow TRK ""Atrium""",0.0,...,0.0,0.0,0.0,0.0,0,0,0,0,0,1
4,0,25,22154,999.0,1.0,23,01,2013,"Moscow TRK ""Atrium""",0.0,...,0.0,0.0,0.0,0.0,0,0,0,0,0,1


# Promotion

### 같은 가게 같은 아이템의 가격의 어느 순간 하락하면 프로모션이 진행되었다고 판단
### 프로모션의 여부 피쳐를 추가해준다.
### 주의. 가격이 상승함은 조건에서 걸러준다

In [47]:
pro_df = df.groupby(["shop_id","item_id","date_block_num","month", "day","year"]).agg({"item_price":"sum"}).unstack().fillna(0)
pro_df.head()

item_price          
year                                           2013 2014 2015
shop_id item_id date_block_num month day                     
0       30      1              02    15       265.0  0.0  0.0
                                     16       265.0  0.0  0.0
                                     17       265.0  0.0  0.0
                                     18       265.0  0.0  0.0
                                     20       265.0  0.0  0.0

In [48]:
pro_df["promotion"] =np.zeros(len(pro_df))
pro_df.head()

item_price           promotion
year                                           2013 2014 2015          
shop_id item_id date_block_num month day                               
0       30      1              02    15       265.0  0.0  0.0       0.0
                                     16       265.0  0.0  0.0       0.0
                                     17       265.0  0.0  0.0       0.0
                                     18       265.0  0.0  0.0       0.0
                                     20       265.0  0.0  0.0       0.0

In [ ]:
# len(df.item_id)

In [ ]:
for i in range(len(df.shop_id)):
    for j in range(len(df.item_id)):
        for k in range(len(df.date_block_num)):
            try:
                if len(set(pro_df['item_price']['2013'][i][j][k])) > 1:
                    pro_df['promotion'][i][j][k] = 1
                    print(i,j,k)
                    print(pro_df['promotion'][i][j][k])
                else:
                    continue
            except:
                pass

In [ ]:
# f = lambda x: x[
# train_df['month'] =list(map(f, pro_df['item_price']['2013']))

In [ ]:
pro_df.to_pickle("./data/pro_df.pickle")

## 상관관계

In [ ]:
df.corrwith(df.item_cnt_day)

In [ ]:
df.corr()

## ==============전처리 끝===================

In [ ]:
x_data = train_df.values

In [ ]:
x_data.shape

In [ ]:
y_data = train_df.item_cnt_day.reshape(train_df.item_cnt_day.size,1)

In [ ]:
from sklearn import preprocessing

minmax_scale = preprocessing.MinMaxScaler(feature_range=(0,5)).fit(x_data)
x_scaled_data = minmax_scale.transform(x_data)

x_scaled_data[:3]

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x_scaled_data, y_data, test_size=0.33)

In [ ]:
y_data.shape

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
from sklearn import  linear_model

regr = linear_model.LinearRegression(n_jobs=1)
regr.fit(X_train, y_train)
regr

In [ ]:
print('Coefficients: ', regr.coef_) # w1
print('intercept: ', regr.intercept_) #w0

# Metric 측정

In [ ]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

In [ ]:
y_true = y_test
y_hat = regr.predict(X_test)

r2_score(y_true, y_hat), mean_absolute_error(y_true, y_hat), mean_squared_error(y_true, y_hat)
# r2_score : 1에 가까우면 좋음  

In [ ]:
y_true = y_train
y_hat = regr.predict(X_train)

r2_score(y_true, y_hat), mean_absolute_error(y_true, y_hat), mean_squared_error(y_true, y_hat)